In [5]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt



# Interactive functions


This provides examples of uses of interactive functions, such as ginput,
waitforbuttonpress and manual clabel placement.

This script must be run interactively using a backend that has a
graphical user interface (for example, using GTK3Agg backend, but not
PS backend).


In [6]:
import time



def tellme(s):
    print(s)
    plt.title(s, fontsize=16)
    plt.draw()

Define a triangle by clicking three points



In [7]:
plt.clf()
plt.setp(plt.gca(), autoscale_on=False)

tellme('You will define a triangle, click to begin')

plt.waitforbuttonpress()

while True:
    pts = []
    while len(pts) < 3:
        tellme('Select 3 corners with mouse')
        pts = np.asarray(plt.ginput(3, timeout=-1))
        if len(pts) < 3:
            tellme('Too few points, starting over')
            time.sleep(1)  # Wait a second

    ph = plt.fill(pts[:, 0], pts[:, 1], 'r', lw=2)

    tellme('Happy? Key click for yes, mouse click for no')

    if plt.waitforbuttonpress():
        break

    # Get rid of fill
    for p in ph:
        p.remove()

You will define a triangle, click to begin
Select 3 corners with mouse
Happy? Key click for yes, mouse click for no
Select 3 corners with mouse
Happy? Key click for yes, mouse click for no


Now contour according to distance from triangle
corners - just an example



In [8]:
# Define a nice function of distance from individual pts
def f(x, y, pts):
    z = np.zeros_like(x)
    for p in pts:
        z = z + 1/(np.sqrt((x - p[0])**2 + (y - p[1])**2))
    return 1/z


X, Y = np.meshgrid(np.linspace(-1, 1, 51), np.linspace(-1, 1, 51))
Z = f(X, Y, pts)

CS = plt.contour(X, Y, Z, 20)

tellme('Use mouse to select contour label locations, middle button to finish')
CL = plt.clabel(CS, manual=True)

Use mouse to select contour label locations, middle button to finish
Select label locations manually using first mouse button.
End manual selection with second mouse button.


Now do a zoom



In [9]:
tellme('Now do a nested zoom, click to begin')
plt.waitforbuttonpress()

while True:
    tellme('Select two corners of zoom, middle mouse button to finish')
    pts = plt.ginput(2, timeout=-1)
    if len(pts) < 2:
        break
    (x0, y0), (x1, y1) = pts
    xmin, xmax = sorted([x0, x1])
    ymin, ymax = sorted([y0, y1])
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

tellme('All Done!')
plt.show()

Now do a nested zoom, click to begin
Select two corners of zoom, middle mouse button to finish
Select two corners of zoom, middle mouse button to finish
Select two corners of zoom, middle mouse button to finish
All Done!


In [10]:

class LineBuilder:
    def __init__(self, line,ax,color):
        self.line = line
        self.ax = ax
        self.color = color
        self.xs = []
        self.ys = []
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)
        self.counter = 0
        self.shape_counter = 0
        self.shape = {}
        self.precision = 10

    def __call__(self, event):
        if event.inaxes!=self.line.axes: return
        if self.counter == 0:
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
        if np.abs(event.xdata-self.xs[0])<=self.precision and np.abs(event.ydata-self.ys[0])<=self.precision and self.counter != 0:
            self.xs.append(self.xs[0])
            self.ys.append(self.ys[0])
            self.ax.scatter(self.xs,self.ys,s=120,color=self.color)
            self.ax.scatter(self.xs[0],self.ys[0],s=80,color='blue')
            self.ax.plot(self.xs,self.ys,color=self.color)
            self.line.figure.canvas.draw()
            self.shape[self.shape_counter] = [self.xs,self.ys]
            self.shape_counter = self.shape_counter + 1
            self.xs = []
            self.ys = []
            self.counter = 0
        else:
            if self.counter != 0:
                self.xs.append(event.xdata)
                self.ys.append(event.ydata)
            self.ax.scatter(self.xs,self.ys,s=120,color=self.color)
            self.ax.plot(self.xs,self.ys,color=self.color)
            self.line.figure.canvas.draw()
            self.counter = self.counter + 1


In [11]:
def create_shape_on_image(data,cmap='jet'):
    def change_shapes(shapes):
        new_shapes = {}
        for i in range(len(shapes)):
            l = len(shapes[i][1])
            new_shapes[i] = np.zeros((l,2),dtype='int')
            for j in range(l):
                new_shapes[i][j,0] = shapes[i][0][j]
                new_shapes[i][j,1] = shapes[i][1][j]
        return new_shapes
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title('click to include shape markers (10 pixel precision to close the shape)')
    line = ax.imshow(data) 
    ax.set_xlim(0,data[:,:,0].shape[1])
    ax.set_ylim(0,data[:,:,0].shape[0])
    linebuilder = LineBuilder(line,ax,'red')
    plt.gca().invert_yaxis()
    plt.show()
    new_shapes = change_shapes(linebuilder.shape)
    return new_shapes

In [14]:
img = np.zeros((100,100,3),dtype='uint')
shapes = create_shape_on_image(img)
print(shapes)

{}


In [21]:
def onclick(event):
    print(event.xdata, event.ydata)
    ax.scatter(event.xdata,event.ydata)

fig,ax = plt.subplots()
ax.imshow(img)
fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

37.05411255411255 46.14502164502164
64.38095238095238 50.47402597402597
72.4978354978355 69.68398268398269
40.84199134199133 78.07142857142857
14.326839826839816 63.19047619047619
4.85714285714284 20.170995670995666


In [22]:

class Cursor(object):
    def __init__(self, ax):
        self.ax = ax
        self.lx = ax.axhline(color='k')  # the horiz line
        self.ly = ax.axvline(color='k')  # the vert line

        # text location in axes coords
        self.txt = ax.text(0.7, 0.9, '', transform=ax.transAxes)

    def mouse_move(self, event):
        if not event.inaxes:
            return

        x, y = event.xdata, event.ydata
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)

        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        self.ax.figure.canvas.draw()


class SnaptoCursor(object):
    """
    Like Cursor but the crosshair snaps to the nearest x, y point.
    For simplicity, this assumes that *x* is sorted.
    """

    def __init__(self, ax, x, y):
        self.ax = ax
        self.lx = ax.axhline(color='k')  # the horiz line
        self.ly = ax.axvline(color='k')  # the vert line
        self.x = x
        self.y = y
        # text location in axes coords
        self.txt = ax.text(0.7, 0.9, '', transform=ax.transAxes)

    def mouse_move(self, event):
        if not event.inaxes:
            return

        x, y = event.xdata, event.ydata
        indx = min(np.searchsorted(self.x, x), len(self.x) - 1)
        x = self.x[indx]
        y = self.y[indx]
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)

        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        print('x=%1.2f, y=%1.2f' % (x, y))
        self.ax.figure.canvas.draw()


t = np.arange(0.0, 1.0, 0.01)
s = np.sin(2 * 2 * np.pi * t)

fig, ax = plt.subplots()
ax.plot(t, s, 'o')
cursor = Cursor(ax)
fig.canvas.mpl_connect('motion_notify_event', cursor.mouse_move)

fig, ax = plt.subplots()
ax.plot(t, s, 'o')
snap_cursor = SnaptoCursor(ax, t, s)
fig.canvas.mpl_connect('motion_notify_event', snap_cursor.mouse_move)

plt.show()

x=0.20, y=0.59
x=0.22, y=0.37
x=0.24, y=0.13
x=0.24, y=0.13
x=0.25, y=0.00
x=0.25, y=0.00
x=0.25, y=0.00
x=0.25, y=0.00
x=0.26, y=-0.13
x=0.29, y=-0.48
x=0.30, y=-0.59
x=0.30, y=-0.59
x=0.32, y=-0.77
x=0.37, y=-1.00
x=0.39, y=-0.98
x=0.41, y=-0.90
x=0.42, y=-0.84
x=0.42, y=-0.84
x=0.42, y=-0.84
x=0.41, y=-0.90
x=0.40, y=-0.95
x=0.39, y=-0.98
x=0.39, y=-0.98
x=0.37, y=-1.00
x=0.34, y=-0.90
x=0.31, y=-0.68
x=0.29, y=-0.48
x=0.27, y=-0.25
x=0.26, y=-0.13
x=0.25, y=0.00
x=0.25, y=0.00
x=0.24, y=0.13
x=0.24, y=0.13
x=0.25, y=0.00
x=0.25, y=0.00
x=0.25, y=0.00
x=0.25, y=0.00
x=0.26, y=-0.13
x=0.27, y=-0.25
x=0.27, y=-0.25
x=0.27, y=-0.25
x=0.27, y=-0.25
x=0.28, y=-0.37
x=0.28, y=-0.37
x=0.28, y=-0.37
x=0.28, y=-0.37
x=0.28, y=-0.37
x=0.29, y=-0.48
x=0.29, y=-0.48
x=0.30, y=-0.59
x=0.30, y=-0.59
x=0.30, y=-0.59
x=0.31, y=-0.68
x=0.31, y=-0.68
x=0.32, y=-0.77
x=0.33, y=-0.84
x=0.34, y=-0.90
x=0.36, y=-0.98
x=0.37, y=-1.00
x=0.38, y=-1.00
x=0.39, y=-0.98
x=0.42, y=-0.84
x=0.44, y=-0.68
x=0.46, 